# Spam Detection 

This is a simple solution of this problem for begginers to introduce data science principles for a easy classification problem. In this `Kernel` we use `Pandas Dataframe` as Data structure and `sklearn` for machine learning stuffs.

### A Data Science Framework



1. **Define the Problem:** If data science, big data, machine learning, predictive analytics, business intelligence, or any other buzzword is the solution, then what is the problem? As the saying goes, don't put the cart before the horse. Problems before requirements, requirements before solutions, solutions before design, and design before technology. Too often we are quick to jump on the new shiny technology, tool, or algorithm before determining the actual problem we are trying to solve.

2. **Gather the Data:** John Naisbitt wrote in his 1984 (yes, 1984) book Megatrends, we are “drowning in data, yet staving for knowledge." So, chances are, the dataset(s) already exist somewhere, in some format. It may be external or internal, structured or unstructured, static or streamed, objective or subjective, etc. As the saying goes, you don't have to reinvent the wheel, you just have to know where to find it. In the next step, we will worry about transforming "dirty data" to "clean data."

3. **Prepare Data for Consumption:** This step is often referred to as data wrangling, a required process to turn “wild” data into “manageable” data. Data wrangling includes implementing data architectures for storage and processing, developing data governance standards for quality and control, data extraction (i.e. ETL and web scraping), and data cleaning to identify aberrant, missing, or outlier data points.

4. **Perform Exploratory Analysis:** Anybody who has ever worked with data knows, garbage-in, garbage-out (GIGO). Therefore, it is important to deploy descriptive and graphical statistics to look for potential problems, patterns, classifications, correlations and comparisons in the dataset. In addition, data categorization (i.e. qualitative vs quantitative) is also important to understand and select the correct hypothesis test or data model.

5. **Model Data:** Like descriptive and inferential statistics, data modeling can either summarize the data or predict future outcomes. Your dataset and expected results, will determine the algorithms available for use. It's important to remember, algorithms are tools and not magical wands or silver bullets. You must still be the master craft (wo)man that knows how-to select the right tool for the job. An analogy would be asking someone to hand you a Philip screwdriver, and they hand you a flathead screwdriver or worst a hammer. At best, it shows a complete lack of understanding. At worst, it makes completing the project impossible. The same is true in data modelling. The wrong model can lead to poor performance at best and the wrong conclusion (that’s used as actionable intelligence) at worst.


6. **Validate and Implement Data Model:** After you've trained your model based on a subset of your data, it's time to test your model. This helps ensure you haven't overfit your model or made it so specific to the selected subset, that it does not accurately fit another subset from the same dataset. In this step we determine if our model overfit, generalize, or underfit our dataset.


7. **Optimize and Strategize:** This is the "bionic man" step, where you iterate back through the process to make it better...stronger...faster than it was before. As a data scientist, your strategy should be to outsource developer operations and application plumbing, so you have more time to focus on recommendations and design. Once you're able to package your ideas, this becomes your “currency exchange" rate.

*Source : [A Data Science Framework: To Achieve 99% Accuracy](https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy)*

## Step1: Define the Problem
In this problem we want to find all spams using classification problems. There is some significant words in spams which can be used to find spams. 

## Step2: Gather the Data
We are using [this](https://www.kaggle.com/uciml/sms-spam-collection-dataset) dataset from kaggle website. This is a collection of SMS messages tagged if they are `spam` or `ham`.

## Step3: Prepare the Data 
I think this step is the most step of these kind of problems. 

### Meet and Greet Data

First we create a repository for our inputs. 

In [ ]:
import os
print(os.listdir("../input")) # For kaggle kernel

As i said we use `Pandas Dataframe` as our data structure. so we import it and load `spam.csv`.

In [ ]:
import pandas as pd 
import numpy as np
DATASET_DIR = '../input'  # For kaggle kernel
# DATASET_DIR = './datasets/spam'
DATASET_NAME = 'spam.csv'
dataset_path = os.path.join(DATASET_DIR, DATASET_NAME)
dataset = pd.read_csv(dataset_path, encoding='ISO-8859-1')

In [ ]:
dataset.head()

### Split TrainSet and TestSet 
It is important to split these two. You always should split test set at the beggining of your solution. At the end of solution you return to test set again and use it just for test!


#### How to split these two ?
Well there is some ways to do that. For example you can select a fraction of your dataset by index (aka. 0 to 1000 indices). Another way which used is random selection. But what if we want to run our thest again?? Well we can select a const random seed. I use `sklearn` for this which used random strategy. 

First we split labels from other datas

In [ ]:
X = dataset.drop('v1', axis=1)
y = dataset['v1']

Then Split them.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

This method also shuffle datas. So you don't need to shuffle them. If you selected train and test sets manually you should shuffle them. Becasue it could be dependancy between datas in a sequence and it impacts on your classification.

### The 4 C's of Data Cleaning: Correcting, Completing, Creating, and Converting

#### Correcting and Completing
We should first review our data and see if there is not to be unaccaptable features and datas. Also there is null values in our data some algorithms can handle null values but it's better to handle these null values.

For this first see our data informations. 

In [ ]:
X_train.info()

**There is a lot's of null values in 3 last columns. We can recognize these columns are not usefull and we can drop them.**

For this goal we write a sklearn transformer to use it with our pipelines in future.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelection(BaseEstimator, TransformerMixin):
    def __init__(self, attrs):
        self.attrs = attrs
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[attrs]

In [ ]:
attrs = ['v2']
selector = DataFrameSelection(attrs)
X = selector.transform(X_train)
X.head()

We should also change labels to a numeric code. We use sklearn LabelEncoder for these purpose.

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
y_train

### Creating
To create new features that can help us to solve this problem we should first analys our data to find out important words. then we can use them to create a `sparse matrix` of existence of these words in message.

First let join X and y to can analys data

In [ ]:
Xy = X
Xy['status'] = y_train
Xy.head()

First See how much of them are spams


In [ ]:
import matplotlib.pyplot as plt
plt.bar(('ham', 'spam'), (len(Xy[Xy['status'] == 0]),len(Xy[Xy['status'] == 1])), color='r')
prob = len(Xy[Xy['status'] == 1])/len(Xy)
print(f"prob of spam is {prob}")

Well there is a lots of ham data in training data

In [ ]:
Xy.groupby('status').describe()

We add length to find out if it is important or not.

In [ ]:
class LengthAdder(BaseEstimator, TransformerMixin):
    def __init__(self, msg_attr='v2'):
        self.msg_attr = msg_attr
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        dataset = X.copy()
        dataset['len'] = X[self.msg_attr].apply(len)
        return dataset

In [ ]:
Xy = LengthAdder().transform(Xy)
Xy.head()

In [ ]:
Xy.hist(column='len', by='status', bins=25, figsize=(15, 7))

From this we can recognize that length of msg can be usefull to find out spam messages.

##### Creating Sparse Matrix
to find out spam messages there is a way to get all possible words in messages and create a sparse matrix of existence of word in message or not. Then train classifier on this sparse matrix.

For that we write a transformer which convert our messages to corresponding sparse matrix.

But before that we should add some hyperparamters. Like converting messages to lowercase, remove punctuation, replace all URLs with `URL`, replace all numbers with `NUMBER`, or even perform stemming.

#### To Lower Case 
if we want to compare words to each other they should all be in lower case. So lets transform it.

In [ ]:
class ToLowerCase(BaseEstimator, TransformerMixin):
    def __init__(self, msg_attr='v2'):
        self.msg_attr = msg_attr
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        dataset = X.copy()
        dataset[self.msg_attr] = dataset[self.msg_attr].str.lower()
        return dataset
Xy = ToLowerCase().transform(Xy)
Xy.head()

#### Delete Stop Words and Punctuations
**stop word** is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore, both when indexing entries for searching and when retrieving them as the result of a search query.



In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
import nltk 
class DeleteStopWordsAndPunc(BaseEstimator, TransformerMixin):
    def __init__(self, lang='english', msg_attr='v2'):
        self.lang = lang
        self.msg_attr = msg_attr
        nltk.download('stopwords')
        self.stop_words = set(stopwords.words('english')) 
        self.punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~+~'''
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        dataset = X.copy()
        dataset[self.msg_attr] = dataset[self.msg_attr].apply(self.delete_stop_words)
        dataset[self.msg_attr] = dataset[self.msg_attr].apply(self.delete_punc)
        return dataset
    def delete_stop_words(self, msg):
        tokens = WhitespaceTokenizer().tokenize(msg)
        return ' '.join([w for w in tokens if w not in self.stop_words])
    def delete_punc(self, msg):
        no_punct = ""
        for char in msg:
            if char not in self.punctuations:
                no_punct = no_punct + char
        return no_punct

Ok. Now let see dataframe **before** this transform :

In [ ]:
Xy.head()

And **after** :

In [ ]:
Xy = DeleteStopWordsAndPunc().transform(Xy)
Xy.head()

#### Stemming Words
Stemming is the process of producing morphological variants of a root/base word. Stemming programs are commonly referred to as stemming algorithms or stemmers. A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”.

Last part of cleaning messages are stemming them. with stemming them we improve chance of finding apearance of a word in messages. To stemming words we can use `nltk` library. It provide us usefull tools to stemming words.

In [ ]:
from nltk.stem import PorterStemmer
class Stemmer(BaseEstimator, TransformerMixin):
    def __init__(self, msg_attr='v2'):
        self.msg_attr = msg_attr
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        dataset = X.copy()
        dataset[self.msg_attr] = dataset[self.msg_attr].apply(self.stemming)
        return dataset
    def stemming(self, msg):
        return ' '.join([PorterStemmer().stem(w) for w in WhitespaceTokenizer().tokenize(msg)])

In [ ]:
Xy = Stemmer().transform(Xy)
Xy.head()

#### Change Numbers and Urls
In this section we change numbers to string `NUMBER` and change urls to `URL`. Because existence of url or number is important for us. Not that number or url itself.


In [ ]:
import re #regex
class ChangeNumbersAndUrls(BaseEstimator, TransformerMixin):
    def __init__(self, msg_attr='v2'):
        self.msg_attr = msg_attr
        self.url_regex = re.compile(
                    r'^(?:http|ftp)s?://' # http:// or https://
                    r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
                    r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
                    r'(?::\d+)?' # optional port
                    r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        dataset = X.copy()
        dataset[self.msg_attr] = dataset[self.msg_attr].apply(self.change_urls_numbers)
        return dataset
    def change_urls_numbers(self, msg):
        tokens = WhitespaceTokenizer().tokenize(msg)
        new_msg = []
        for w in tokens:
            if w.isnumeric() :
                new_msg.append('NUMBER')
            elif re.match(self.url_regex, w):
                new_msg.append('URL')
            else : new_msg.append(w)
        return ' '.join(new_msg)

In [ ]:
Xy = ChangeNumbersAndUrls().transform(Xy)
Xy.head()

### Converting 
Last, but certainly not least, we'll deal with formatting. For this problem we said that we want to make a matrix of existence of every word in message. 

* First we should find all seperate words
* Then fill all words existence in matrix

In [ ]:
class CreateExistenceMatrix(BaseEstimator, TransformerMixin):
    def __init__(self, msg_attr='v2', len_index=2):
        self.word_set = set({})
        self.msg_attr = msg_attr
        self.len_index = len_index
    def fit(self, X, y=None):
        for m in X[self.msg_attr]: 
            for w in WhitespaceTokenizer().tokenize(m):
                self.word_set.add(w)
        return self
    def transform(self, X, y=None):
        dataset = X.copy()
        for word in self.word_set:
            dataset[word] = 0
        indices = dataset.columns[self.len_index:]
        for index, row in dataset.iterrows():
            tokens = WhitespaceTokenizer().tokenize(row[self.msg_attr])
            for word in tokens:
                if word in dataset.columns:
                    dataset.at[index, word] = dataset.at[index, word] + 1
        return dataset
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X, y)

In [ ]:
Xy = CreateExistenceMatrix(len_index=3).fit_transform(Xy)

### Conver to Sparse Matrix 
Sparse matrix just contains vlues which is not zero. And there is lots of algorithms which support sparse matrix. So we can have a very faster way to classify our big matrix.

In [ ]:
from scipy.sparse import csr_matrix
class ToSparseMatrix(BaseEstimator, TransformerMixin):
    def __init__(self, msg_attr='v2'):
        self.msg_attr = msg_attr
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return csr_matrix(X.drop(self.msg_attr, axis=1).values)
Xy = ToSparseMatrix().transform(Xy)

## Create Pipeline
sklearn pipelines help us to set a pipeline for all of our process. Here we make a pipeline to process all we did to here in a single pipeline

In [ ]:
from sklearn.pipeline import Pipeline
attrs = ['v2']
full_pipeline = Pipeline([
    ('selector', DataFrameSelection(attrs)),
    ('length_adder', LengthAdder()),
    ('to_lower', ToLowerCase()),
    ('delete_stop_punc', DeleteStopWordsAndPunc()),
    ('stemming', Stemmer()),
    ('change_number_url', ChangeNumbersAndUrls()),
    ('create_existance', CreateExistenceMatrix()),
    ('to_sparse', ToSparseMatrix()),
])

In [ ]:
X_train_sparse = full_pipeline.fit_transform(X_train)

## Modeling
Now lets do the magic. Here we select some classificaiton models. For this i use RandomForest model from tree models, SGD model from linear models and some Navie Bayes modles. We examine all models and select some of them to optimise and make them better. 


Our error scoring use precision, recall and f1_score. If you don't know what they are you can find usefull informations [here](https://en.wikipedia.org/wiki/Precision_and_recall). 

First we define all of our models in an array. 


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
models = [
    ('svc', SVC(kernel='rbf')),
    ('neighbors', KNeighborsClassifier(3)),
    ('random_forest', RandomForestClassifier()),
    ('sgd', SGDClassifier()), 
    ('mutlinomial_nb', MultinomialNB()),
    ('complement_nb', ComplementNB()),
    ('bernoli_nb', BernoulliNB()),
]

Then compute every score for all of them.

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, recall_score, precision_score, precision_recall_curve, roc_curve
scores = pd.DataFrame([], columns=['model_name', 'f1', 'precision', 'recall'])
for model in models:
    predictions = cross_val_predict(model[1], X_train_sparse, y_train, cv=3, n_jobs=-1)
    scores = scores.append({
        'model_name':model[0],
        'f1': f1_score(y_train, predictions),
        'precision': precision_score(y_train, predictions),
        'recall': recall_score(y_train, predictions),
    }, ignore_index=True)

In [ ]:
scores

## Optimise and Strategize 
Now we should see what is important for us. Well in spam problem it is important to block all spams and recognize not spam messages. But we can say it is more important to block all spams. So we can say `precision` and `recall` are both important. but `recall` is more important than `precision`.

So if model have a low f1 it is not good for us. it also should have a recall more than 90. 

Ok. We select `multinomial navie bayes`. Also we can select `random forest` because it has a high precision and we can sacrifise it to get more recall. 

### Try to Select Best Model
We know we can change precision to have better recall. For that we see all possible probablity thresholds for every model to select best threshold. 

So we plot recall precisions with various thresholds.

In [ ]:
def show_plot_precision_recall(precision, recall, threshold, model_name, ax):
    ax.plot(threshold, precision[:-1], "b--", label="Precision")
    ax.plot(threshold, recall[:-1], "r-", label="Recall")
    ax.set_xlabel("Threshold")
    ax.set_ylabel(model_name)
    ax.legend(loc="lower right")
    ax.set_ylim([0, 1])

In [ ]:
def draw_models_score_plot(models):
    i = 1
    fig = plt.figure(figsize=(20, 15))
    fig.subplots_adjust(hspace=0.4)
    for model in models:
        ax = fig.add_subplot(3, 3, i)
        i+= 1
        y_scores = []
        if hasattr(model[1], "decision_function"):
            y_scores = cross_val_predict(model[1], X_train_sparse, y_train, cv=3, method="decision_function", n_jobs=-1)
        else :
            y_probs = cross_val_predict(model[1], X_train_sparse, y_train, cv=3, method="predict_proba", n_jobs=-1)
            y_scores = y_probs[:, -1]
        precision, recall, thresholds = precision_recall_curve(y_train, y_scores)
        show_plot_precision_recall(precision, recall, thresholds, model[0], ax)
draw_models_score_plot(models) 


From these plots we recognize we should select one of navie bayes.

These kind of models have an aplha parameters which is very important. So let select three of them.

In [ ]:
models = [
    ('mutlinomial_nb_0.2', MultinomialNB(alpha=0.2)),
    ('mutlinomial_nb_0.5', MultinomialNB(alpha=0.5)),
    ('mutlinomial_nb_1', MultinomialNB(alpha=1.0)),
    ('complement_nb_0.2', ComplementNB(alpha=0.2)),
    ('complement_nb_0.5', ComplementNB(alpha=0.5)),
    ('complement_nb_1', ComplementNB(alpha=1)),
    ('bernoli_nb_0.2', BernoulliNB(alpha=0.2)),
    ('bernoli_nb_0.5', BernoulliNB(alpha=0.5)),
    ('bernoli_nb_1', BernoulliNB(alpha=1)),
]
draw_models_score_plot(models)

As you see we can select a bernoli navie bayes with low alpha to have great score in both recall and precision. 

### Try to Select Best HyperParameters 
For this goal we use grid search. It means try some possible hyperparameters of model but using sklearn tools. 

We use recall scoring. because recall is more important for us.

In [ ]:
from sklearn.model_selection import GridSearchCV
estimator = BernoulliNB()
grid_params = {
    'alpha': [0, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2],
    'fit_prior': [True, False],
}
grid_search = GridSearchCV(estimator, grid_params, scoring='recall')
grid_search.fit(X_train_sparse, y_train)
grid_search.best_score_

In [ ]:
best_estimator = grid_search.best_estimator_
best_estimator

Now lets select a good threshold to have both good recall and precision. 

In [ ]:
y_probs = cross_val_predict(best_estimator, X_train_sparse, y_train, cv=3, method="predict_proba", n_jobs=-1)
y_scores = y_probs[:, -1]
precision, recall, thresholds = precision_recall_curve(y_train, y_scores)
ax = plt.axes()
show_plot_precision_recall(precision, recall, thresholds, "Bernoli NB", ax)

Well we can select threshold now. I select 0.4 threshold. Because it have a better recall and good precision. We should see what we need. Here recall is more important for me.

In [ ]:
def predict(estimator, X, threshold=0.5):
    y_probs = estimator.predict_proba(X)[:,1]
    return (y_probs >= threshold)

In [ ]:
X_test_transformed = full_pipeline.transform(X_test)

In [ ]:
test_predictions = predict(best_estimator, X_test_transformed, 0.4)
recall = recall_score(y_test, test_predictions)
precision = precision_score(y_test, test_predictions)
f1 = f1_score(y_test, test_predictions)
print(f'Precision is {precision}\nRecall is {recall}\nF1 Score is {f1}')